In [2]:
# Initial imports
import os
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
! pip install alpaca_trade_api
from pandas.tseries.offsets import DateOffset
from alpaca_trade_api import TimeFrame
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
# import plotly.graph_objects as go
# import plotly.express as px
import numpy as np
# import hvplot.pandas
from sklearn.ensemble import RandomForestClassifier
# pd.options.plotting.backend = 'plotly'
# Load .env environment variables
load_dotenv()

#TODO
# add columns to each input to the model
# which cover the past week(adjustable range) of price and volatility

True

In [3]:
import alpaca_trade_api as tradeapi

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
alpaca_base_url = "https://paper-api.alpaca.markets"

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

ValueError: ('Key ID must be given to access Alpaca trade API', ' (env: APCA_API_KEY_ID)')

In [ ]:
# Create a connection to the API 
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, alpaca_base_url, api_version="v2")


In [ ]:
tesla = 'TSLA'
start_date = '2016-01-01'
end_date = '2022-10-19'

timeframe = TimeFrame.Day

In [ ]:
SMA_fast = 5
SMA_slow = 13

In [ ]:
tesla_data = api.get_bars(tesla, timeframe, start_date, end_date).df
# tesla_data.plot()

In [ ]:
# tesla_data = tesla_data.filter(['close'])
# tesla_data.rename(columns={'close':'TSLA'}, inplace = True)
# tesla_data.index = tesla_data.index.map(lambda timestamp : timestamp.date)

In [ ]:
tesla_data

In [ ]:
tesla_data = tesla_data.filter(['close'])
tesla_data.rename(columns={'close':'TSLA'}, inplace = True)
tesla_data.index = tesla_data.index.map(lambda timestamp : timestamp.date)
tesla_data['slow_SMA'] = tesla_data['TSLA'].rolling(window=SMA_slow).mean()
tesla_data['fast_SMA'] = tesla_data['TSLA'].rolling(window=SMA_fast).mean()

In [ ]:
X = tesla_data[['fast_SMA', 'slow_SMA']].shift().dropna().copy()

In [ ]:
tesla_data['signal'] = 0.0
tesla_data['actual_returns'] = tesla_data['TSLA'].pct_change()
tesla_data

In [ ]:
for i in tesla_data['actual_returns']:
    if i > 0:
        z = True
    else:
        z = False

In [ ]:
tesla_data.loc[(tesla_data['actual_returns'] >= 0), 'signal'] = -1
tesla_data.loc[(tesla_data['actual_returns'] < 0), 'signal'] = 1

In [ ]:
y = tesla_data['signal'].copy()

In [ ]:
training_begin = X.index.min()
print(training_begin)
training_end = X.index.min() + DateOffset(months=72)


In [ ]:
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

In [ ]:
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

In [ ]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

In [ ]:
rf_model = rf_model.fit(X_train_scaled, y_train)

In [ ]:
predictions = rf_model.predict(X_test_scaled)

In [ ]:
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

In [ ]:

# Create a Dataframe using predicted data
predictions_data = pd.DataFrame(index=X_test.index)
testing_report = classification_report(y_test, predictions)
print(testing_report)

# Add two new columns to the DateFrame called predicted signals and actual returns
predictions_data['predicted_signals'] = predictions
predictions_data['actual_returns'] = tesla_data['actual_returns']

# Add third column that is the sum of the two previously made columns
predictions_data['predictions'] = (predictions_data['actual_returns'] * predictions_data['predicted_signals'])

In [ ]:

(1 + predictions_data[['actual_returns', 'predictions']]).cumprod().plot()